In [11]:
## Script to do some data transformations

#Import Statements

import pandas as pd
import numpy as np
import json
import requests
pd.set_option('display.min_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
#Pull the data from csv file

game_data_df = pd.read_csv('2019_game_data.csv')  

game_data_df['away_win'] = ~game_data_df['home_win']

#Unique teams
game_data_df['home_team_id'].unique()

array([10, 19, 22, 54, 14,  3,  5, 12, 18, 25, 21, 24,  1,  2, 29,  4, 28,
        7,  9, 13, 15, 53, 20, 17, 23,  8, 52, 16, 26,  6, 30],
      dtype=int64)

In [3]:
#Loop through each team

cnt = 0 

for team_id in game_data_df['home_team_id'].unique():

    print(team_id)

    #Find home and away games
    
    home_games = game_data_df[game_data_df['home_team_id']==team_id]
    away_games = game_data_df[game_data_df['away_team_id']==team_id]

    all_games = pd.concat([home_games, away_games])
    all_games = all_games.sort_values(by=['game_id'])

    #all_games
    
    all_games.reset_index(inplace=True, drop=True)

    # For each game take the previous 20 games and average over all stats for home team
    #all_games.shape[0]

    #display(all_games.head())
    
    #Average over past 20 games

    data_averaged = []
    for n in np.arange(20,all_games.shape[0]):

        #print(n)

        games_20 = all_games.iloc[np.arange(n-20,n),:]
        #print(games_20)    

        data = []    

        for index, row in games_20.iterrows():

            #print(row)

            if row['home_team_id'] == team_id:
                data2 = list(row[games_20.columns[games_20.columns.str.match('home')]].values) 
            else:
                data2 = list(row[games_20.columns[games_20.columns.str.match('away')]].values) 

            data.append(data2)

        col_list = list(games_20.columns[games_20.columns.str.match('home')]) 

        #Place into dataframe
        games_20_df = pd.DataFrame(data,columns=col_list)
        
        #Take average of previous 20 games on all predictors
        games_20_df_avg = list(games_20_df.mean().values)
    
        if all_games.iloc[n,:]['home_team_id'] == team_id:
            games_20_df_avg = [all_games.iloc[n,:]['game_id']] + games_20_df_avg + [ 1 ] + [all_games.iloc[n,:]['home_win']]
        else:
            games_20_df_avg = [all_games.iloc[n,:]['game_id']] + games_20_df_avg + [ 0 ] + [all_games.iloc[n,:]['home_win']]

        data_averaged.append(games_20_df_avg)
        
        col_replace = [w.replace('home', 'team') for w in col_list[1:]]
        
        col_list_2 =  ['game_number'] + col_replace + ['is_home_team'] + ['home_victory']

    if cnt == 0: 
        games_predict = pd.DataFrame(data_averaged,columns=col_list_2)
    else:
        games_predict_new  = pd.DataFrame(data_averaged,columns=col_list_2)

        games_predict = pd.concat([games_predict, games_predict_new])

    cnt = cnt + 1 

#Predictors for each game for the Leafs


10
19
22
54
14
3
5
12
18
25
21
24
1
2
29
4
28
7
9
13
15
53
20
17
23
8
52
16
26
6
30


In [12]:
#Sort games by game number

games_predict.sort_values(['game_number'])


,game_number,team_team_id,team_goals,team_pim,team_shots,team_powerPlayPercentage,team_powerPlayGoals,team_powerPlayOpportunities,team_faceOffWinPercentage,team_blocked,team_takeaways,team_giveaways,team_hits,team_SV_Pct,team_SH_Pct,team_CF_Pct,team_win,is_home_team,home_victory
0,2019020284,20.0,2.85,9.95,31.20,14.910,0.60,3.30,48.920,12.90,8.35,10.75,14.85,0.906477,0.087778,0.498103,0.45,1,False
1,2019020290,22.0,3.00,7.85,28.75,24.995,0.80,2.85,48.170,15.00,8.80,12.10,21.55,0.916649,0.105589,0.483323,0.65,1,True
2,2019020293,17.0,2.30,8.70,30.35,14.635,0.55,3.30,49.195,13.80,4.35,12.75,19.55,0.884089,0.070861,0.506476,0.35,0,True
3,2019020294,10.0,3.35,7.90,33.30,14.545,0.60,3.40,52.350,12.75,7.30,12.55,19.35,0.903873,0.100786,0.513725,0.45,1,False
4,2019020296,15.0,3.85,9.25,31.40,25.680,0.80,3.50,49.850,15.40,7.55,10.00,22.55,0.907076,0.117306,0.515198,0.70,1,False
5,2019020300,52.0,2.65,4.65,33.15,12.830,0.50,3.00,51.165,11.60,7.70,10.70,24.10,0.908473,0.081445,0.507429,0.50,0,False
6,2019020301,22.0,3.15,8.20,29.05,28.330,1.00,3.05,48.175,14.90,8.60,12.00,21.55,0.916649,0.109485,0.487836,0.65,1,False
7,2019020301,25.0,2.50,8.10,29.70,10.880,0.40,2.95,51.970,14.05,6.80,12.45,17.40,0.921241,0.080722,0.497560,0.50,0,False
8,2019020302,53.0,2.75,6.65,31.05,16.165,0.50,3.10,49.120,12.20,7.40,9.50,17.75,0.920472,0.091320,0.480060,0.55,1,True
9,2019020302,20.0,2.75,8.80,30.95,13.660,0.55,3.30,48.865,12.70,8.85,11.25,14.50,0.909703,0.084778,0.497508,0.45,0,True


In [5]:
#Output model data

games_predict.to_csv('team_model_2019.csv',index=False)

# Next
# Place game duplicates on same line, train test split, parameter tune, run logistic regression model